### 導入套件 + 設定路徑

In [6]:
import pandas as pd
from pathlib import Path
import glob
import datetime
import re
import os
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from datetime import date
import math

In [7]:
# home and time
home = Path.home()
todaystr = date.today().strftime('%Y-%m-%d')

# set up concat directories
targetFolder = Path(home, 'HP Inc','GPSTW SOP - 2021 日新','Project team','Upload folder ( for buyer update )')
FD_folder = Path(targetFolder, "FD_today","*.xlsx")
shortage_folder = Path(targetFolder ,"shortage_today","*.xlsx")
PNbasedDetail_folder = Path(targetFolder ,"PNbasedDetail_today","*.xlsx")

### FD 合併以及整理

#### 合併檔案

In [8]:
# concat FD
FD_files = []
for FD_file in glob.glob(str(Path(FD_folder))):
    print(FD_file)
    FD = pd.read_excel(FD_file)
    FD_files.append(FD)
FD_all = pd.concat(FD_files)

C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221121_GPU_CEI_Jennifer_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221121_GPU_CQIEC_Jennifer_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221121_GPU_CQQCI_Jennifer_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221121_LCD_CQIEC_WangYu-Ting_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221121_LCD_CQQCI_WangYu-Ting_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221122_AC ADAPTOR_CQIEC_ChenJosephine(GPS)_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221122_AC ADAPTOR_CQIEC_HsiehAngela_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP 

In [9]:
# remember inplace = True
FD_all['len_FV'] = FD_all['FV'].str.len()
FD_all['len_platform'] = FD_all['Platform'].str.len()
FD_all = FD_all.sort_values( ['len_FV','len_platform'] , ascending = [False,False] )
FD_all.reset_index( drop = True, inplace = True )

#### 將max len放到前方

In [10]:
# Index where B is longest
FD_idmax = FD_all['len_platform'].max()
df_max = FD_all.loc[FD_all['len_platform'] == FD_idmax]
df_max_to_add_FD = df_max.head(1)

In [11]:
# isin用在list
# drop the max len row
FD_all = FD_all.drop([FD_all.index[df_max_to_add_FD.index.values[0]]]).reset_index( drop = True )

In [12]:
# concat and put the max len row on the top
FD_concat = [df_max_to_add_FD,FD_all]
FD_output = pd.concat(FD_concat)
FD_output.reset_index( drop = True , inplace = True)

In [13]:
# cut more than 500
FD_output['Platform'] = FD_output['Platform'].apply(lambda x: x[:450] if len(x) > 500 else x)
FD_output = FD_output.drop( columns = ['len_FV','len_platform'])
FD_output['Item'] = FD_output['Item'].astype(str)

### Shortage 合併以及整理

#### 合併檔案

In [14]:
# concat shortage
shortage_files = []
for shortage_file in glob.glob(str(Path(shortage_folder))):
    print(shortage_file)
    shortage = pd.read_excel(shortage_file)
    shortage_files.append(shortage)
shortage_all = pd.concat(shortage_files)

C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20221121_GPU_CEI_Jennifer_Shortage.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20221121_GPU_CQIEC_Jennifer_Shortage.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20221121_GPU_CQQCI_Jennifer_Shortage.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20221121_LCD_CQIEC_WangYu-Ting_Shortage.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20221121_LCD_CQQCI_WangYu-Ting_Shortage.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20221122_AC ADAPTOR_CQIEC_ChenJosephine(GPS)_Shortage.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today

In [15]:
# create value to sort 
shortage_all['len_FV'] = shortage_all['FV'].str.len()
shortage_all['len_platform'] = shortage_all['Platform'].str.len()
shortage_all = shortage_all.sort_values(['len_FV','len_platform'] , ascending=[False,False])
shortage_all.reset_index( drop = True, inplace = True )

#### 將max len放到前面

In [16]:
# find the max length 
shortage_idmax = shortage_all['len_platform'].max()
shortage_max = shortage_all.loc[shortage_all['len_platform'] == shortage_idmax]
df_max_to_add_shortage = shortage_max.head(1)
df_max_to_add_shortage

,ODM,Item,Commodity,FV,Platform,HP_PN,P1,Net P2,Net P3,Total Shortage Qty,BT shortage,Working on upside,ReportDate,last FD date,BuyerName,len_FV,len_platform
179,CQQCI,14.0,CONNECTOR,CONN DDR4 260p STD H4,Colts 1.0/Rebak 20C1/Rebak21C1/Langkawi 20C2/L...,"821112-AS4,821112-LS4,821112-FN4,821112-DS5",0,0,170000,170000,0,0,2022-11-22,2022-11-29 00:00:00,LiangLavinia,21,541


In [17]:
# drop the max len row
shortage_all = shortage_all.drop([shortage_all.index[df_max_to_add_shortage.index.values[0]]]).reset_index( drop = True )

In [18]:
# concat and put the max len row on the top
shortage_concat = [ df_max_to_add_shortage , shortage_all ]
shortage_output = pd.concat(shortage_concat)
shortage_output.reset_index( drop = True , inplace = True)

In [19]:
# cut more than 500
shortage_output['Platform'] = shortage_output['Platform'].apply(lambda x: x[:450] if len(x) > 500 else x)
shortage_output = shortage_output.drop( columns= ['len_FV','len_platform'])
shortage_output['Item'] = shortage_output['Item'].astype(str)

### PNbasedDetail 合併及整理

#### 合併檔案

In [20]:
# concat PNbasedDetail
PNbasedDetail_files = []
for PNbasedDetail_file in glob.glob(str(Path(PNbasedDetail_folder))):
    print(PNbasedDetail_file)
    PNbasedDetail = pd.read_excel(PNbasedDetail_file)
    PNbasedDetail_files.append(PNbasedDetail)
PNbasedDetail_all = pd.concat(PNbasedDetail_files)

C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20221121_GPU_CEI_Jennifer_PNbasedDetail.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20221121_GPU_CQIEC_Jennifer_PNbasedDetail.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20221121_GPU_CQQCI_Jennifer_PNbasedDetail.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20221121_LCD_CQQCI_WangYu-Ting_PNbasedDetail.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20221122_AC ADAPTOR_CQIEC_ChenJosephine(GPS)_PNbasedDetail.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20221122_AC ADAPTOR_CQIEC_HsiehAngela_PNbasedDetail.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 202

In [21]:
# create list to sort 
sort_list = []
asc_list = []

try:
    PNbasedDetail_all['len_GPS Remark'] = PNbasedDetail_all['GPS Remark'].str.len()
    sort_list.append('len_GPS Remark')
    asc_list.append(True)
except:    
    pass
try:
    PNbasedDetail_all['len_ODM use column1'] = PNbasedDetail_all['ODM use column1'].str.len()
    sort_list.append('len_ODM use column1')
    asc_list.append(True)
except:    
    pass
try:
    PNbasedDetail_all['len_ODM use column2'] = PNbasedDetail_all['ODM use column2'].str.len()
    sort_list.append('len_ODM use column2')
    asc_list.append(True)
except:    
    pass
try:
    PNbasedDetail_all['len_ODM use column3'] = PNbasedDetail_all['ODM use column3'].str.len()
    sort_list.append('len_ODM use column3')
    asc_list.append(True)
except:    
    pass
try:
    PNbasedDetail_all['len_ODM use column4'] = PNbasedDetail_all['ODM use column4'].str.len()
    sort_list.append('len_ODM use column4')
    asc_list.append(True)
except:    
    pass
try:
    PNbasedDetail_all['len_ODM use column5'] = PNbasedDetail_all['ODM use column5'].str.len()
    sort_list.append('len_ODM use column5')
    asc_list.append(True)
except:    
    pass

#### 將max len放到前面

In [22]:
# sort value
PNbasedDetail_all = PNbasedDetail_all.sort_values( sort_list , ascending = asc_list )
PNbasedDetail_all.reset_index( drop=True , inplace=True ) 

In [23]:
# find the max length & concat
PNbasedDetail_max_files = []

# a for loop to calculate all the len max
for i in range( 1 , len(sort_list) ):
    PNbasedDetail_idmax = PNbasedDetail_all [ sort_list[i] ].max()
    PNbasedDetail_max = PNbasedDetail_all.loc[ PNbasedDetail_all[ sort_list[i] ] == PNbasedDetail_idmax ]
    PNbasedDetail_max_files.append( PNbasedDetail_max.head(1) )
    df_max_to_add_PNbasedDetail_temp = pd.concat( PNbasedDetail_max_files )

# sometimes with duplicates 
df_max_to_add_PNbasedDetail = df_max_to_add_PNbasedDetail_temp.drop_duplicates()

In [24]:
# check point
df_max_to_add_PNbasedDetail

,ODM,Item,Commodity,HP PN,GPS Remark,852 stock,852 stock change,Over pull qty,ODM use column1,ODM use column2,ODM use column3,ODM use column4,ODM use column5,ReportDate,BuyerName,len_GPS Remark,len_ODM use column1,len_ODM use column2,len_ODM use column3
90,CQQCI,14.0,CONNECTOR,821112-AS4,NaN,94609,0,0,/0P1B/0P5F/0P5J/0P5H/0P5N/0P5M/0P9C/0P9F/0PAA/...,Colts 1.0/Rebak 20C1/Rebak21C1/Langkawi 20C2/L...,NaN,NaN,NaN,2022-11-22,LiangLavinia,NaN,215.0,541.0,NaN
8,CQQCI,4.0,LCD,L72970-S91,NaN,0,0,0,0GS,14 FHD AG 400N-2ANT-Total,L72970-J93 PE hold on AMD platform,NaN,NaN,2022-11-21,WangYu-Ting,NaN,3.0,25.0,34.0


In [25]:
# drop the max len row
for i in range( 0, len(df_max_to_add_PNbasedDetail.index.values)):
    PNbasedDetail_all = PNbasedDetail_all.drop([PNbasedDetail_all.index[df_max_to_add_PNbasedDetail.index.values[i]]])

In [26]:
# concat and put on the top
PNbasedDetail_concat_list = [ df_max_to_add_PNbasedDetail , PNbasedDetail_all ]
PNbasedDetail_output = pd.concat(PNbasedDetail_concat_list).reset_index( drop = True )
PNbasedDetail_output

,ODM,Item,Commodity,HP PN,GPS Remark,852 stock,852 stock change,Over pull qty,ODM use column1,ODM use column2,ODM use column3,ODM use column4,ODM use column5,ReportDate,BuyerName,len_GPS Remark,len_ODM use column1,len_ODM use column2,len_ODM use column3
0,CQQCI,14.0,CONNECTOR,821112-AS4,NaN,94609,0,0,/0P1B/0P5F/0P5J/0P5H/0P5N/0P5M/0P9C/0P9F/0PAA/...,Colts 1.0/Rebak 20C1/Rebak21C1/Langkawi 20C2/L...,NaN,NaN,NaN,2022-11-22,LiangLavinia,NaN,215.0,541.0,NaN
1,CQQCI,4.0,LCD,L72970-S91,NaN,0,0,0,0GS,14 FHD AG 400N-2ANT-Total,L72970-J93 PE hold on AMD platform,NaN,NaN,2022-11-21,WangYu-Ting,NaN,3.0,25.0,34.0
2,WHFXN,1.0,POWERCORDS,L97622-105,SAB(sea shipment) cut off 11/23,0,0,0,SAB cut off 11/23,NaN,NaN,NaN,NaN,2022-11-22,ChaoAngie(GPS),31.0,17.0,NaN,NaN
3,FLH,4,SSS,L94564-001,pls work with WHFXN for internal SB,0,0,0,NaN,NaN,NaN,NaN,NaN,2022-11-22,LaiShirley,35.0,NaN,NaN,NaN
4,CQQCI,19,GCHIPSET,L47216-001,NaN,0,0,0,F93,Drogo,NaN,NaN,NaN,2022-11-21,Jennifer,NaN,3.0,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,WHFXN,57,USBIC,M08634-001,NaN,0,0,4800,NaN,NaN,NaN,NaN,NaN,2022-11-22,WangAlice(GPSTaiwan),NaN,NaN,NaN,NaN
293,WHFXN,58,USBIC,L84438-002,NaN,0,0,1786,NaN,NaN,NaN,NaN,NaN,2022-11-22,WangAlice(GPSTaiwan),NaN,NaN,NaN,NaN
294,WHFXN,59,USBIC,N04486-002,NaN,0,0,2500,NaN,NaN,NaN,NaN,NaN,2022-11-22,WangAlice(GPSTaiwan),NaN,NaN,NaN,NaN
295,WHFXN,60,USBIC,L11510-001,NaN,0,0,18786,NaN,NaN,NaN,NaN,NaN,2022-11-22,WangAlice(GPSTaiwan),NaN,NaN,NaN,NaN


In [27]:
# cut more than 500
try:
    PNbasedDetail_output['GPS Remark'] = PNbasedDetail_output['GPS Remark'].apply(lambda x: x[:450] if len(x) > 500 else x)
except:    
    pass
try:
    PNbasedDetail_output['ODM use column1'] = PNbasedDetail_output['ODM use column1'].apply(lambda x: x[:450] if len(x) > 500 else x)
except:    
    pass
try:
    PNbasedDetail_output['ODM use column2'] = PNbasedDetail_output['ODM use column2'].apply(lambda x: x[:450] if len(x) > 500 else x)
except:    
    pass
try:
    PNbasedDetail_output['ODM use column3'] = PNbasedDetail_output['ODM use column3'].apply(lambda x: x[:450] if len(x) > 500 else x)
except:    
    pass
try:
    PNbasedDetail_output['ODM use column4'] = PNbasedDetail_output['ODM use column4'].apply(lambda x: x[:450] if len(x) > 500 else x)
except:    
    pass
try:
    PNbasedDetail_output['ODM use column5'] = PNbasedDetail_output['ODM use column5'].apply(lambda x: x[:450] if len(x) > 500 else x)
except:    
    pass


In [28]:
# final step, drop calculate step and output
PNbasedDetail_output = PNbasedDetail_output.drop( columns = sort_list )
PNbasedDetail_output['Item'] = PNbasedDetail_output['Item'].astype(str)
PNbasedDetail_output

,ODM,Item,Commodity,HP PN,GPS Remark,852 stock,852 stock change,Over pull qty,ODM use column1,ODM use column2,ODM use column3,ODM use column4,ODM use column5,ReportDate,BuyerName
0,CQQCI,14.0,CONNECTOR,821112-AS4,NaN,94609,0,0,/0P1B/0P5F/0P5J/0P5H/0P5N/0P5M/0P9C/0P9F/0PAA/...,Colts 1.0/Rebak 20C1/Rebak21C1/Langkawi 20C2/L...,NaN,NaN,NaN,2022-11-22,LiangLavinia
1,CQQCI,4.0,LCD,L72970-S91,NaN,0,0,0,0GS,14 FHD AG 400N-2ANT-Total,L72970-J93 PE hold on AMD platform,NaN,NaN,2022-11-21,WangYu-Ting
2,WHFXN,1.0,POWERCORDS,L97622-105,SAB(sea shipment) cut off 11/23,0,0,0,SAB cut off 11/23,NaN,NaN,NaN,NaN,2022-11-22,ChaoAngie(GPS)
3,FLH,4,SSS,L94564-001,pls work with WHFXN for internal SB,0,0,0,NaN,NaN,NaN,NaN,NaN,2022-11-22,LaiShirley
4,CQQCI,19,GCHIPSET,L47216-001,NaN,0,0,0,F93,Drogo,NaN,NaN,NaN,2022-11-21,Jennifer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,WHFXN,57,USBIC,M08634-001,NaN,0,0,4800,NaN,NaN,NaN,NaN,NaN,2022-11-22,WangAlice(GPSTaiwan)
293,WHFXN,58,USBIC,L84438-002,NaN,0,0,1786,NaN,NaN,NaN,NaN,NaN,2022-11-22,WangAlice(GPSTaiwan)
294,WHFXN,59,USBIC,N04486-002,NaN,0,0,2500,NaN,NaN,NaN,NaN,NaN,2022-11-22,WangAlice(GPSTaiwan)
295,WHFXN,60,USBIC,L11510-001,NaN,0,0,18786,NaN,NaN,NaN,NaN,NaN,2022-11-22,WangAlice(GPSTaiwan)


### FD、Shortage、PNbasedDetail 輸出檔案

In [29]:
# apache airflow to upload SQL ( currently to desktop )
FD_output.to_excel(Path(home, 'Desktop', 'FD_all.xlsx'), index=False)
shortage_output.to_excel(Path(home, 'Desktop', 'Shortage_all.xlsx'), index=False)
PNbasedDetail_output.to_excel(Path(home, 'Desktop', 'PNbasedDetail_all.xlsx'), index=False)